In [14]:
import pandas as pd
import os
from pandas import DataFrame as df

In [15]:
# import the two csv files needed
#STEP1 : Import the csv file containing the admin boundaries in level 2 and the production in ha per district
path = r"C:\Oluchi\Benin\SNV DATA"
name_of_file = "production_maize.csv"
# Import csv as pandas dataframe
production_file= pd.read_csv(path + "\\" + name_of_file)
production_file.head(3)

,name_adm2,production_tonnes
0,Abomey,3031
1,Abomey-calavi,12116
2,Adja-ouere,25045


In [16]:
#Step2: Import the csv file containing the harvested area in level 2 according to MAP SPAM. This is the test output from the 
#: #downscaling process
path = r"C:\Benin\agrodem_preprocessing\Downscaling\Output_Data\Crop_Maps_modified"
name_of_file = "Ben_Maize_test.csv"
# Import csv as pandas dataframe
input= pd.read_csv(path + "\\" + name_of_file)
input.head(3)

,Unnamed: 0,iso3,prod_level,alloc_key,cell5m,x,y,maiz_a,name_cntr,name_adm1,name_adm2,Admin_2_share,Area_Harv_2018
0,0,BEN,BN12001,9942184,4291943,1.958333,7.208333,7010.9,Benin,Zou,Abomey,0.006454,7472.628755
1,1,BEN,BN03001,10022188,4326507,2.291667,6.541667,3320.0,Benin,Atlantique,Abomey-calavi,0.003056,3538.650882
2,2,BEN,BN03001,10022189,4326508,2.375000,6.541667,952.7,Benin,Atlantique,Abomey-calavi,0.000877,1015.443583


In [17]:
# STEP3 : Drop all columns except the crop harvest area in 2018  and the district names colums and create a new dataframe
new= input.drop(['Unnamed: 0', 'iso3', 'prod_level', 'alloc_key', 'cell5m', 'x', 'y',
                  'maiz_a', 'name_cntr', 'name_adm1','Admin_2_share'],axis = 1)

In [18]:
# STEP4: Calculate the sum of the harvested area in each district (admin level 2)
new= new.groupby('name_adm2').sum()

In [19]:
# STEP 5: rename the column as required 
new.rename(columns={'Area_Harv_2018':'Harvest_Area_By_District'}, inplace=True)

In [20]:
#STEP6: merge the new dataframe to the initial input file
new_input = new.merge(input,on = 'name_adm2')

In [21]:
#STEP7: merge the production column (which is a seperate dataframe) with the output of the previous step 
new_input_final= new_input.merge(production_file,on ='name_adm2', how = 'left')

In [22]:
new_input_final['ratio_harvested_area_by_district']= new_input_final['Area_Harv_2018']/new_input_final['Harvest_Area_By_District']

In [23]:
new_input_final['final_prod_tonnes']=new_input_final['ratio_harvested_area_by_district']*new_input_final['production_tonnes']

In [24]:
new_input_final.head(-3)

,name_adm2,Harvest_Area_By_District,Unnamed: 0,iso3,prod_level,alloc_key,cell5m,x,y,maiz_a,name_cntr,name_adm1,Admin_2_share,Area_Harv_2018,production_tonnes,ratio_harvested_area_by_district,final_prod_tonnes
0,Abomey,7472.628755,0,BEN,BN12001,9942184,4291943,1.958333,7.208333,7010.9,Benin,Zou,0.006454,7472.628755,3031.0,1.000000,3031.000000
1,Abomey-calavi,9780.532597,1,BEN,BN03001,10022188,4326507,2.291667,6.541667,3320.0,Benin,Atlantique,0.003056,3538.650882,12116.0,0.361806,4383.635928
2,Abomey-calavi,9780.532597,2,BEN,BN03001,10022189,4326508,2.375000,6.541667,952.7,Benin,Atlantique,0.000877,1015.443583,12116.0,0.103823,1257.918659
3,Abomey-calavi,9780.532597,3,BEN,BN03001,10032188,4330827,2.291667,6.458333,2513.3,Benin,Atlantique,0.002314,2678.822669,12116.0,0.273893,3318.491620
4,Abomey-calavi,9780.532597,4,BEN,BN03001,10032189,4330828,2.375000,6.458333,1334.7,Benin,Atlantique,0.001229,1422.601606,12116.0,0.145452,1762.300865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1159,Zogbodome,14856.684652,1159,BEN,BN12009,9962186,4300585,2.125000,7.041667,1645.8,Benin,Zou,0.001515,1754.190247,NaN,0.118074,NaN
1160,Zogbodome,14856.684652,1160,BEN,BN12009,9962187,4300586,2.208333,7.041667,1321.4,Benin,Zou,0.001216,1408.425685,NaN,0.094801,NaN
1161,Zogbodome,14856.684652,1161,BEN,BN12009,9962188,4300587,2.291667,7.041667,943.7,Benin,Zou,0.000869,1005.850855,NaN,0.067704,NaN
1162,Zogbodome,14856.684652,1162,BEN,BN12009,9972185,4304904,2.041667,6.958333,1101.4,Benin,Zou,0.001014,1173.936771,NaN,0.079017,NaN


In [25]:
x=new_input_final['final_prod_tonnes'].sum()
print(x)

2276192.9999999995


In [26]:
path = r"C:\Benin\agrodem_preprocessing\Downscaling\Output_Data\Crop_Maps_modified"

new_input_final.to_csv(os.path.join(path,"{c}.csv".format(c="Ben_Maize_final")))